In [267]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import talib
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import seaborn as sns

import optuna
from optuna.samplers import TPESampler
from optuna.trial import TrialState
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau 
import shap
import plotly.graph_objs as go
import plotly.offline as pyo
from tqdm.auto import tqdm
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F
from tqdm.notebook import tqdm


from sklearn.model_selection import KFold
import copy
from torch.cuda.amp import GradScaler, autocast
import os


In [268]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")
else:
    device = torch.device('cpu')
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN version:', torch.backends.cudnn.version())


gpu
2.1.2+cu121
CUDA available: True
CUDA version: 12.1
cuDNN version: 8902


In [269]:
pivoted_data = pd.read_csv("../Data/longmerged_deneme_51.csv")
pivoted_data.drop(columns=["Unnamed: 0"], inplace=True)
#pivoted_data.set_index("PENDS", inplace=True)
len(pivoted_data["OFTIC"].unique())
# pivoted_data[pivoted_data["OFTIC"] == "AAPL"]
pivoted_data = pivoted_data[pivoted_data['PENDS'] > '2011-03-31']
pivoted_data = pivoted_data[pivoted_data['PENDS'] < '2020-01-01']

pivoted_data

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
0,2011-06-30,AAPL,2011-06-30,17.869831,1.698451,2.6718,0.422900,777.000000,0.209015,0.00,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
1,2011-09-30,AAPL,2011-09-30,17.834875,3.131120,2.9872,0.397600,1645.000000,0.258659,0.00,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
2,2011-12-31,AAPL,2011-12-31,21.771714,3.087308,3.4500,0.665700,1321.000000,0.249095,0.00,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
3,2012-03-31,AAPL,2012-03-31,26.426070,2.380069,3.9154,0.528200,1457.000000,0.153558,0.00,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
4,2012-06-30,AAPL,2012-06-30,27.652509,3.109565,4.2582,0.384300,2056.000000,0.204900,0.00,...,0.484614,-2.905304,2.024644,-7.498254,-7.341768,1.063541,-4.676644,-4.741380,-4.544226,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2018-12-31,ZION,2018-12-31,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.30,...,0.355782,-15.534891,-5.840908,-24.280433,-13.633068,-9.080399,-17.844385,-17.722026,-12.791300,0.512814
14056,2019-03-31,ZION,2019-03-31,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.30,...,-0.178828,14.988381,10.496262,15.292074,7.934506,6.057101,16.488114,19.393354,9.857481,9.920635
14057,2019-06-30,ZION,2019-06-30,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.30,...,0.081598,4.699164,3.493137,-3.644893,7.351231,0.970027,3.185392,5.459461,5.405405,2.509890
14058,2019-09-30,ZION,2019-09-30,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.34,...,-0.036101,1.258389,5.768897,-7.078949,1.449274,-2.709415,0.271247,3.190669,-0.512819,8.569506


In [270]:
unique_counts = pivoted_data.groupby('Sector')['OFTIC'].nunique()
unique_counts

Sector
Communication Services     5
Consumer Discretionary    32
Consumer Staples          20
Energy                    14
Financials                38
Health Care               38
Industrials               36
Information Technology    34
Materials                 19
Real Estate               20
Utilities                 20
Name: OFTIC, dtype: int64

In [271]:
pivoted_data.columns

Index(['Date', 'OFTIC', 'PENDS', 'MEAN', 'STDEV', 'BPS', 'CPS', 'CPX', 'CSH',
       'DPS', 'EBG', 'EBI', 'EBS', 'EBT', 'ENT', 'EPS', 'FFO', 'GPS', 'GRM',
       'NAV', 'NDT', 'NET', 'OPR', 'PRE', 'ROA', 'ROE', 'SAL',
       'Real_Estate_Index_Price', 'VIX_Close', 'Gold_Close',
       'Three_Month_Yield', 'Brent_Close', 'Hrc_close',
       'commodity_trade_Close', 'C_Discretionary_Close', 'C_Staples_Close',
       'Energy_Close', 'Financials_Close', 'Health_care_Close',
       'industrials_Close', 'information_Close', 'materials_Close',
       'utilities_Close', 'Sector', 'numeric_sector',
       'Real_Estate_Index_Price_quarterly_return',
       'VIX_Close_quarterly_return', 'Gold_Close_quarterly_return',
       'Three_Month_Yield_quarterly_return', 'Brent_Close_quarterly_return',
       'Hrc_close_quarterly_return', 'commodity_trade_Close_quarterly_return',
       'C_Discretionary_Close_quarterly_return',
       'C_Staples_Close_quarterly_return', 'Energy_Close_quarterly_return',
   

In [272]:
pivoted_data = pivoted_data[~pivoted_data["OFTIC"].isin(["DFS", "ICE"])]

In [273]:
pivoted_data = pivoted_data[pivoted_data["Sector"] == "Financials"]
pivoted_data

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
51,2011-06-30,AFL,2011-06-30,27.546538,3.568238,12.8250,1.587282,0.259769,0.235599,0.150,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
52,2011-09-30,AFL,2011-09-30,26.911765,2.682322,13.6250,1.763514,0.324273,0.310576,0.150,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
53,2011-12-31,AFL,2011-12-31,27.750000,1.982881,14.4800,1.621515,0.320066,0.322786,0.165,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
54,2012-03-31,AFL,2012-03-31,28.115385,2.550767,14.5950,1.660416,0.201505,0.197983,0.165,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
55,2012-06-30,AFL,2012-06-30,28.200000,2.592189,15.1450,1.823892,0.289115,0.257621,0.165,...,0.484614,-2.905304,2.024644,-7.498254,-7.341768,1.063541,-4.676644,-4.741380,-4.544226,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2018-12-31,ZION,2018-12-31,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.300,...,0.355782,-15.534891,-5.840908,-24.280433,-13.633068,-9.080399,-17.844385,-17.722026,-12.791300,0.512814
14056,2019-03-31,ZION,2019-03-31,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.300,...,-0.178828,14.988381,10.496262,15.292074,7.934506,6.057101,16.488114,19.393354,9.857481,9.920635
14057,2019-06-30,ZION,2019-06-30,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.300,...,0.081598,4.699164,3.493137,-3.644893,7.351231,0.970027,3.185392,5.459461,5.405405,2.509890
14058,2019-09-30,ZION,2019-09-30,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.340,...,-0.036101,1.258389,5.768897,-7.078949,1.449274,-2.709415,0.271247,3.190669,-0.512819,8.569506


In [274]:
pivoted_data[pivoted_data["OFTIC"] == "ACN"]

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return


In [275]:
pivoted_data = pivoted_data.copy()

pivoted_data['Hrc_close'] = pd.to_numeric(pivoted_data['Hrc_close'].astype(str).str.split(",").str[0], errors='coerce')
pivoted_data['Gold_Close'] = pd.to_numeric(pivoted_data['Gold_Close'].astype(str).str.replace(",", ""), errors='coerce').round(2)

In [276]:
company_list = pivoted_data["OFTIC"].unique()
company_list

array(['AFL', 'AON', 'AXP', 'BEN', 'BK', 'BLK', 'BX', 'CBOE', 'CMA',
       'CME', 'COF', 'GS', 'HIG', 'IVZ', 'KEY', 'MA', 'MCO', 'MKTX',
       'MMC', 'MSCI', 'MTB', 'NDAQ', 'NTRS', 'PFG', 'PGR', 'PNC', 'PRU',
       'RF', 'RJF', 'SCHW', 'STT', 'TROW', 'TRV', 'WFC', 'WRB', 'ZION'],
      dtype=object)

In [277]:
pivoted_data.shape

(1260, 61)

In [278]:
company_list = pivoted_data["OFTIC"].unique()
company_list

array(['AFL', 'AON', 'AXP', 'BEN', 'BK', 'BLK', 'BX', 'CBOE', 'CMA',
       'CME', 'COF', 'GS', 'HIG', 'IVZ', 'KEY', 'MA', 'MCO', 'MKTX',
       'MMC', 'MSCI', 'MTB', 'NDAQ', 'NTRS', 'PFG', 'PGR', 'PNC', 'PRU',
       'RF', 'RJF', 'SCHW', 'STT', 'TROW', 'TRV', 'WFC', 'WRB', 'ZION'],
      dtype=object)

In [279]:
pivoted_data[pivoted_data["OFTIC"] == "AAPL"].sort_values(by="PENDS")

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return


In [280]:
cols_to_move = ['OFTIC', 'PENDS', 'Date', 'Sector', 'numeric_sector']
pivoted_data = pivoted_data[[col for col in pivoted_data if col not in cols_to_move] + cols_to_move]
pivoted_data

,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,EBG,EBI,EBS,...,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return,OFTIC,PENDS,Date,Sector,numeric_sector
51,27.546538,3.568238,12.8250,1.587282,0.259769,0.235599,0.150,0.227910,1367.020851,0.251326,...,7.308964,-1.141483,-1.381423,-1.624192,5.051769,AFL,2011-06-30,2011-06-30,Financials,5
52,26.911765,2.682322,13.6250,1.763514,0.324273,0.310576,0.150,0.319596,1389.471671,0.315255,...,-10.695185,-21.535988,-8.171207,-25.425447,0.418158,AFL,2011-09-30,2011-09-30,Financials,5
53,27.750000,1.982881,14.4800,1.621515,0.320066,0.322786,0.165,0.269770,1109.000000,0.361496,...,9.328708,15.503083,7.838985,14.100815,7.019633,AFL,2011-12-31,2011-12-31,Financials,5
54,28.115385,2.550767,14.5950,1.660416,0.201505,0.197983,0.165,0.183399,1382.000000,0.199331,...,8.417417,10.874069,18.506872,10.358213,-2.612559,AFL,2012-03-31,2012-03-31,Financials,5
55,28.200000,2.592189,15.1450,1.823892,0.289115,0.257621,0.165,0.248831,1221.000000,0.282720,...,1.063541,-4.676644,-4.741380,-4.544226,5.565071,AFL,2012-06-30,2012-06-30,Financials,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.300,0.519314,297.000000,0.587800,...,-9.080399,-17.844385,-17.722026,-12.791300,0.512814,ZION,2018-12-31,2018-12-31,Financials,5
14056,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.300,0.245076,278.000000,0.327376,...,6.057101,16.488114,19.393354,9.857481,9.920635,ZION,2019-03-31,2019-03-31,Financials,5
14057,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.300,0.489922,277.000000,0.540476,...,0.970027,3.185392,5.459461,5.405405,2.509890,ZION,2019-06-30,2019-06-30,Financials,5
14058,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.340,0.638738,298.000000,0.654236,...,-2.709415,0.271247,3.190669,-0.512819,8.569506,ZION,2019-09-30,2019-09-30,Financials,5


In [281]:
stock_symbol = "AFL"
stock = pivoted_data[pivoted_data["OFTIC"] == stock_symbol].sort_values(by="PENDS")

stock_features_dict = {}
for column in stock.columns:
    stock_features_dict[column] = stock[column]

trace = go.Scatter(x=stock_features_dict["PENDS"], y=stock_features_dict["EPS"], mode="lines+markers", name=f"{stock_symbol}: EPS - PENDS")

layout = go.Layout(
    title = f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=trace, layout=layout)
pyo.iplot(fig)

In [282]:
X_ = pivoted_data.drop(columns=["EPS"])
y_ = pivoted_data['EPS']
y_

51       0.780
52       0.830
53       0.740
54       0.825
55       0.805
         ...  
14055    1.080
14056    1.040
14057    0.990
14058    1.170
14059    0.970
Name: EPS, Length: 1260, dtype: float64

In [283]:
x_values = pivoted_data.drop(columns=["OFTIC","PENDS","Date","Sector","numeric_sector"])
x_values

,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,EBG,EBI,EBS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
51,27.546538,3.568238,12.8250,1.587282,0.259769,0.235599,0.150,0.227910,1367.020851,0.251326,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
52,26.911765,2.682322,13.6250,1.763514,0.324273,0.310576,0.150,0.319596,1389.471671,0.315255,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
53,27.750000,1.982881,14.4800,1.621515,0.320066,0.322786,0.165,0.269770,1109.000000,0.361496,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
54,28.115385,2.550767,14.5950,1.660416,0.201505,0.197983,0.165,0.183399,1382.000000,0.199331,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
55,28.200000,2.592189,15.1450,1.823892,0.289115,0.257621,0.165,0.248831,1221.000000,0.282720,...,0.484614,-2.905304,2.024644,-7.498254,-7.341768,1.063541,-4.676644,-4.741380,-4.544226,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.300,0.519314,297.000000,0.587800,...,0.355782,-15.534891,-5.840908,-24.280433,-13.633068,-9.080399,-17.844385,-17.722026,-12.791300,0.512814
14056,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.300,0.245076,278.000000,0.327376,...,-0.178828,14.988381,10.496262,15.292074,7.934506,6.057101,16.488114,19.393354,9.857481,9.920635
14057,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.300,0.489922,277.000000,0.540476,...,0.081598,4.699164,3.493137,-3.644893,7.351231,0.970027,3.185392,5.459461,5.405405,2.509890
14058,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.340,0.638738,298.000000,0.654236,...,-0.036101,1.258389,5.768897,-7.078949,1.449274,-2.709415,0.271247,3.190669,-0.512819,8.569506


In [284]:
corr = x_values.corr()

fig = go.Figure(data=go.Heatmap(
                   z=corr.values,
                   x=corr.columns,
                   y=corr.columns,
                   text=corr.round(2).values,
                   texttemplate="%{text}",
                   colorscale='Viridis',
                   zmin=-1, zmax=1))

# Update the layout
fig.update_layout(
    title='Correlation Matrix',
    xaxis_title="Variables",
    yaxis_title="Variables",
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange='reversed'),
    width=900,  # or any width you desire
    height=800,  # or any height you desire
)

fig.show()

In [285]:
linked = sch.linkage(sch.distance.pdist(corr), method='ward')
cluster_order = sch.dendrogram(linked, no_plot=True)['leaves']

# Reorder the correlation matrix
correlation_matrix_ordered = corr.iloc[cluster_order, cluster_order]

fig = go.Figure(data=go.Heatmap(
                   z=correlation_matrix_ordered.values,
                   x=correlation_matrix_ordered.columns,
                   y=correlation_matrix_ordered.columns,
                   text=correlation_matrix_ordered.round(2).values,
                   texttemplate="%{text}",
                   colorscale='Viridis',
                   zmin=-1, zmax=1))

# Update the layout
fig.update_layout(
    title='Correlation Matrix',
    xaxis_title="Variables",
    yaxis_title="Variables",
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange='reversed'),
    width=900,  # or any width you desire
    height=800,  # or any height you desire
)

fig.show()

In [286]:
cluster_order += [max(cluster_order)+1, max(cluster_order)+2, max(cluster_order)+3,max(cluster_order)+4,max(cluster_order)+5]
cluster_order

[7,
 13,
 30,
 27,
 39,
 32,
 35,
 37,
 38,
 31,
 34,
 36,
 44,
 49,
 50,
 52,
 54,
 43,
 48,
 53,
 47,
 51,
 46,
 25,
 41,
 33,
 26,
 28,
 3,
 0,
 1,
 22,
 15,
 21,
 24,
 40,
 55,
 29,
 45,
 5,
 42,
 23,
 16,
 10,
 19,
 18,
 8,
 20,
 6,
 2,
 12,
 14,
 9,
 4,
 11,
 17,
 56,
 57,
 58,
 59,
 60]

In [287]:
pivoted_data = pivoted_data.iloc[:, cluster_order]
pivoted_data

,EBG,FFO,commodity_trade_Close,Three_Month_Yield,utilities_Close,C_Staples_Close,Health_care_Close,information_Close,materials_Close,C_Discretionary_Close,...,GPS,EBS,CPX,ENT,NDT,OFTIC,PENDS,Date,Sector,numeric_sector
51,0.227910,0.225153,99.675354,0.03,33.480000,31.230000,35.529999,25.700001,39.369999,40.209999,...,0.300,0.251326,0.259769,0.219455,0.234633,AFL,2011-06-30,2011-06-30,Financials,5
52,0.319596,0.317883,99.795883,0.02,33.619999,29.660000,31.730000,23.600000,29.360001,34.860001,...,0.795,0.315255,0.324273,0.303188,0.304899,AFL,2011-09-30,2011-09-30,Financials,5
53,0.269770,0.327487,99.709015,0.02,35.980000,32.490002,34.689999,25.450001,33.500000,39.020000,...,0.585,0.361496,0.320066,0.336828,0.295398,AFL,2011-12-31,2011-12-31,Financials,5
54,0.183399,0.199184,99.517723,0.07,35.040001,34.080002,37.610001,30.160000,36.970001,45.090000,...,0.840,0.199331,0.201505,0.175010,0.190412,AFL,2012-03-31,2012-03-31,Financials,5
55,0.248831,0.264272,100.000000,0.09,36.990002,34.770000,38.009998,28.730000,35.290001,43.779999,...,0.515,0.282720,0.289115,0.249700,0.242842,AFL,2012-06-30,2012-06-30,Financials,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,0.519314,0.566079,100.758179,2.45,52.919998,50.779999,86.510002,61.980000,50.520000,99.010002,...,1.080,0.587800,45.779944,0.609228,0.525924,ZION,2018-12-31,2018-12-31,Financials,5
14056,0.245076,0.294405,100.577995,2.40,58.169998,56.110001,91.750000,74.000000,55.500000,113.849998,...,1.040,0.327376,41.655270,0.333021,0.296725,ZION,2019-03-31,2019-03-31,Financials,5
14057,0.489922,0.538019,100.660065,2.12,59.630001,58.070000,92.639999,78.040001,58.500000,119.199997,...,0.990,0.540476,44.643278,0.548109,0.526172,ZION,2019-06-30,2019-06-30,Financials,5
14058,0.638738,0.665659,100.623726,1.88,64.739998,61.419998,90.129997,80.529999,58.200001,120.699997,...,1.170,0.654236,44.768947,0.726207,0.641649,ZION,2019-09-30,2019-09-30,Financials,5


In [288]:
company_dict = {}

for company in tqdm(pivoted_data["OFTIC"].unique()):
    comp_data = pivoted_data[pivoted_data["OFTIC"] == company].sort_values(by='PENDS')
    X = comp_data.drop(columns=["EPS", "Sector","PENDS","OFTIC","Date"])
    y = comp_data['EPS']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    X_train_df = X_train
    X_test_df = X_test
    y_train_df = y_train
    y_test_df = y_test

    for column in X_train.columns:
        if "embedding" in column:
            continue

        scaler = MinMaxScaler()

        X_train_scaled = scaler.fit_transform(X_train[[column]].values)
        X_train_df[column] = X_train_scaled
            
        X_test_scaled = scaler.transform(X_test[[column]].values)
        X_test_df[column] = X_test_scaled

    scaler_y = MinMaxScaler()

    y_train_scaled = pd.DataFrame(scaler_y.fit_transform(y_train.values.reshape(-1, 1)), columns=['EPS'], index=y_train.index)
    y_train_df = y_train_scaled
            
    y_test_scaled = pd.DataFrame(scaler_y.transform(y_test.values.reshape(-1, 1)), columns=['EPS'], index=y_test.index)
    y_test_df = y_test_scaled

    X_train_df["lagged_EPS"] = y_train_df
    X_test_df["lagged_EPS"] = y_test_df

    company_dict[company] = {"X_train": X_train_df, "X_test": X_test_df, "y_train": y_train_df, "y_test": y_test_df, "scaler_y": scaler_y}


  0%|          | 0/36 [00:00<?, ?it/s]

In [289]:
company_dict["AFL"]["X_test"]

,EBG,FFO,commodity_trade_Close,Three_Month_Yield,utilities_Close,C_Staples_Close,Health_care_Close,information_Close,materials_Close,C_Discretionary_Close,...,PRE,DPS,BPS,GPS,EBS,CPX,ENT,NDT,numeric_sector,lagged_EPS
79,0.826128,0.864740,0.596333,1.115607,0.944302,0.803158,1.015309,1.096844,0.921078,1.120578,...,0.639713,1.000000,0.980024,0.287850,0.717296,0.820201,0.833997,0.835628,0.0,1.106667
80,1.207215,1.264188,0.552220,1.265896,0.979561,0.891296,1.244750,1.236968,0.916587,1.239801,...,0.446015,1.000000,0.953405,0.295327,1.184915,1.151386,1.198384,1.184623,0.0,1.026667
81,1.002111,1.101017,0.775552,1.416185,0.993357,0.775615,1.075172,0.917743,0.678858,0.965678,...,0.407785,1.000000,0.986609,0.145794,1.076053,1.010111,1.112325,1.060511,0.0,1.000000
82,0.444183,0.248053,0.662898,1.387283,1.261625,0.971355,1.178018,1.205165,0.838627,1.189071,...,0.644810,1.090909,1.196537,0.347664,0.253698,0.452812,0.240911,0.476706,0.0,1.266667
83,0.887430,0.975364,0.714209,1.225434,1.336229,1.043335,1.195486,1.301770,0.934873,1.269607,...,0.652456,1.090909,1.369652,0.295327,0.930325,0.965583,0.913918,0.976911,0.0,1.293333
84,1.218511,1.296393,0.691490,1.086705,1.597343,1.166361,1.146222,1.361310,0.925249,1.292187,...,0.685588,1.090909,1.472723,0.276636,1.356457,1.291918,1.303241,1.365598,0.0,1.373333
85,0.850800,1.028198,0.651317,0.895954,1.591211,1.223650,1.376447,1.627690,1.028553,1.363239,...,0.351714,1.090909,1.440552,0.284112,0.984323,0.929323,0.978454,0.957014,0.0,1.026667


In [290]:
company_dict["AFL"]["X_train"].shape

(28, 57)

In [291]:
company_dict["AFL"]["y_test"]

,EPS
79,1.106667
80,1.026667
81,1.000000
82,1.266667
83,1.293333
84,1.373333
85,1.026667


In [292]:
# Custom Dataset
class RollingWindowDataset(Dataset):
    def __init__(self, X, y, device=device):
        self.X = X.clone().detach().to(torch.float)
        self.y = y.clone().detach().to(torch.float)

    def __len__(self):
        return len(self.X) 

    def __getitem__(self, idx):
        # Ensure idx is within the valid range
        if idx > len(self.X):
            raise IndexError("Index out of bounds")

        X_window = self.X[idx]
        y_target = self.y[idx]  

        return X_window.clone().detach().to(torch.float).to(device), y_target.clone().detach().to(torch.float).to(device)

In [293]:
X_train = []
y_train = []

X_test = []
y_test = []

time_steps = 4
print(len(company_dict[company]["X_train"]))
print((len(company_dict[company]["X_test"])))

for company in company_list:
    company_dict[company]["y_train"]["Type"] = "Train"
    company_dict[company]["y_test"]["Type"] = "Test"

    comp_df_X = pd.concat([company_dict[company]['X_train'], company_dict[company]['X_test']], axis=0)
    comp_df_y = pd.concat([company_dict[company]['y_train'], company_dict[company]['y_test']], axis=0)

    for i in range((len(comp_df_X)) - time_steps):

        if comp_df_y.iloc[i + time_steps]["Type"] == "Train":
            X_train.append(comp_df_X.iloc[i : (i + time_steps)])
            y_train.append(comp_df_y.iloc[i + time_steps]["EPS"])

        elif comp_df_y.iloc[i + time_steps]["Type"] == "Test":
            X_test.append(comp_df_X.iloc[i : (i + time_steps)])
            y_test.append(comp_df_y.iloc[i + time_steps]["EPS"])

    # for i in range((len(company_dict[company]["X_train"])) - time_steps):

    #     X_train.append(company_dict[company]["X_train"].iloc[i : (i + time_steps)])
    #     y_train.append(company_dict[company]["y_train"][i+time_steps])

    # for i in range((len(company_dict[company]["X_test"])) - time_steps):

    #     X_test.append(company_dict[company]["X_test"].iloc[i : (i+ time_steps)])
    #     y_test.append(company_dict[company]["y_test"][i+time_steps])

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

28
7


In [294]:
y_test

array([ 1.10666667,  1.02666667,  1.        ,  1.26666667,  1.29333333,
        1.37333333,  1.02666667,  0.44736842,  0.26754386,  0.64473684,
        1.14912281,  0.51754386,  0.33333333,  0.80701754,  0.78688525,
        0.81967213,  0.70491803,  0.92622951,  0.97540984,  0.98360656,
        0.94262295,  0.37209302,  1.25581395, -0.11627907,  0.30232558,
        0.13953488,  0.04651163,  0.25581395,  0.89705882,  0.94117647,
        0.83823529,  0.76470588,  0.86764706,  0.95588235,  0.86764706,
        0.98966408,  1.2118863 ,  0.83979328,  0.97674419,  0.9250646 ,
        1.11627907,  1.42377261,  0.72674419,  0.64534884,  0.23255814,
        0.45930233,  0.53488372,  0.54069767,  0.62209302,  0.67647059,
        0.68627451,  1.15686275,  0.73529412,  0.75490196,  0.91176471,
        0.83333333,  1.23387097,  1.2983871 ,  1.24193548,  1.42741935,
        1.29032258,  1.30645161,  1.21774194,  0.90243902,  0.66666667,
        0.92682927,  0.80487805,  0.91869919,  1.03252033,  0.72

In [295]:
X_test.shape

(252, 4, 57)

In [296]:
X_train.shape

(864, 4, 57)

In [297]:
X_train

array([[[0.21216573, 0.17762852, 0.09855325, ..., 0.26082965,
         0.        , 0.36      ],
        [0.43480938, 0.43147686, 0.17390993, ..., 0.44958088,
         0.        , 0.49333333],
        [0.31381473, 0.45776731, 0.11959856, ..., 0.4240608 ,
         0.        , 0.25333333],
        [0.10407803, 0.10654103, 0.        , ..., 0.14203851,
         0.        , 0.48      ]],

       [[0.43480938, 0.43147686, 0.17390993, ..., 0.44958088,
         0.        , 0.49333333],
        [0.31381473, 0.45776731, 0.11959856, ..., 0.4240608 ,
         0.        , 0.25333333],
        [0.10407803, 0.10654103, 0.        , ..., 0.14203851,
         0.        , 0.48      ],
        [0.26296843, 0.28471819, 0.30152688, ..., 0.28287934,
         0.        , 0.42666667]],

       [[0.31381473, 0.45776731, 0.11959856, ..., 0.4240608 ,
         0.        , 0.25333333],
        [0.10407803, 0.10654103, 0.        , ..., 0.14203851,
         0.        , 0.48      ],
        [0.26296843, 0.28471819, 0.3

In [298]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float, device=device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float, device=device)

train_data = RollingWindowDataset(X_train_tensor, y_train_tensor, device=device)
test_data = RollingWindowDataset(X_test_tensor, y_test_tensor, device=device)

print(test_data.__getitem__(0)[1])
print(test_data.__getitem__(1)[0])


tensor(1.1067, device='cuda:0')
tensor([[1.0000, 1.0000, 0.7656, 0.6127, 1.0000, 0.8931, 0.9814, 0.8489, 0.8803,
         0.8313, 0.8881, 0.8995, 0.9171, 0.6872, 0.6252, 0.6960, 0.7838, 0.2940,
         0.2677, 0.6063, 0.4784, 0.5388, 0.1143, 0.0000, 0.1488, 0.2397, 0.3073,
         0.2367, 0.6459, 0.7432, 0.4825, 0.1056, 1.0000, 0.4363, 0.4033, 0.7087,
         0.4117, 0.4444, 0.2943, 1.0000, 0.6628, 0.2579, 0.7923, 0.8129, 1.0000,
         0.3800, 0.5816, 0.3262, 0.5909, 0.8156, 0.2243, 1.0000, 1.0000, 1.0000,
         1.0000, 0.0000, 0.5467],
        [0.6338, 0.5133, 0.6857, 0.8035, 0.9811, 1.0000, 1.0000, 0.9648, 1.0000,
         0.9609, 1.0000, 1.0000, 0.8576, 0.6881, 0.6948, 0.7590, 0.8094, 0.3747,
         0.6006, 0.6139, 0.7922, 0.4632, 0.1207, 0.0457, 0.2997, 0.3306, 0.3398,
         0.3457, 0.3846, 0.9142, 0.6524, 0.0000, 0.4722, 0.2462, 0.4785, 0.7957,
         0.2806, 0.5791, 0.6245, 0.5145, 0.6308, 0.2125, 1.0000, 0.5283, 0.4351,
         0.2480, 0.0000, 0.2447, 0.6818, 0.

In [299]:
# class OneDimCNNLSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob, conv_channels, kernel_size, pool_size, stride):
#         super(OneDimCNNLSTMModel, self).__init__()

#         self.hidden_size = hidden_size
#         self.layer_size = layer_size
#         self.hn, self.cn = None, None

#         conv_output_width = (input_dim - kernel_size)  + 1
#         # Pooling output width (no padding, considering the stride for pooling)
#         pool_output_width = (conv_output_width - pool_size) // stride + 1

#         self.lstm_input_size = conv_channels * pool_output_width  # LSTM input dimensions

#         self.conv = nn.Conv2d(in_channels=1, out_channels=conv_channels, kernel_size=(1, kernel_size))
#         self.relu1 = nn.ReLU()
#         self.maxpool = nn.MaxPool2d(kernel_size=(1,pool_size), stride=(1, stride))

#         self.lstm = nn.LSTM(input_size = self.lstm_input_size, hidden_size = self.hidden_size, num_layers=self.layer_size,
#                             dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
#         self.dropout = nn.Dropout(dropout_prob)
        
#         self.fc = nn.Linear(self.hidden_size, output_dim)

#     def init_hidden(self, batch_size):
#         # Initialize hidden and cell states with zeros
#         h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         return (h0, c0)
    
#     def reset_hidden(self):
#         self.hn = None
#         self.cn = None

#     def forward(self, x):

#         batch_size, seq_len, num_of_feature = x.shape
#         x = x.view(batch_size, 1, seq_len, num_of_feature)

#         x = self.conv(x)
#         x = self.relu1(x)
#         x = self.maxpool(x)

#         batch_size, channels, height, width = x.shape
#         x = x.permute(0, 2, 3, 1) # [batch, height, width, channels]
#         x = x.reshape(batch_size, height*width, channels)
#         # print(x.shape)

#         assert x.size(-1) == self.lstm.input_size, f"Mismatch in LSTM input size. Expected: {self.lstm.input_size}, Got: {x.size(-1)}"

#         if self.hn == None or self.cn == None:
#             self.hn, self.cn = self.init_hidden(x.size(0))
        
#         else:
#             self.hn, self.cn = self.hn.detach(), self.cn.detach()
            
#         if self.hn.size(1) > x.size(0):
#             self.hn = self.hn[:, -x.size(0):, :].contiguous()
#             self.cn = self.cn[:, -x.size(0):, :].contiguous()

#         # Forward propagate LSTM
#         out, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))

#         out = self.dropout(out[:, -1, :])  # Add dropout

#         out = self.fc(out)

#         return out

In [300]:
class OneDimCNNLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob, conv_channels, kernel_size):
        super(OneDimCNNLSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.layer_size = layer_size
        self.hn, self.cn = None, None
        self.padding = 2

        # 1D Convolution
        self.conv = nn.Conv1d(in_channels=input_dim, out_channels=conv_channels, kernel_size=kernel_size, padding=self.padding)
        self.relu1 = nn.ReLU()
        # self.maxpool = nn.MaxPool1d(kernel_size=pool_size, stride=stride)
        self.dropout_1 = nn.Dropout(dropout_prob)

        # Calculate output width after convolution and pooling
        conv_output_length = (input_dim - kernel_size + 2 * self.padding) + 1
        # pool_output_length = (conv_output_length - pool_size) // stride + 1

        self.lstm_input_size = conv_channels   # LSTM input dimensions

        # LSTM Layer
        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=hidden_size, num_layers=layer_size,
                            dropout=(dropout_prob if layer_size > 1 else 0), batch_first=True)
        self.dropout_2 = nn.Dropout(dropout_prob)
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size, output_dim)

    def init_hidden(self, batch_size):
        device = next(self.parameters()).device
        h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size, device=device)
        c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size, device=device)
        return (h0, c0)

    def reset_hidden(self):
        self.hn, self.cn = None, None

    def forward(self, x):
        batch_size, seq_len, num_features = x.shape

        # Permute x to match [batch_size, num_features, seq_len]
        x = x.permute(0, 2, 1)

        # Apply 1D convolution
        x = self.conv(x)
        x = self.relu1(x)
        x = self.dropout_1(x)

        # Prepare for LSTM
        # Assuming output of conv is [batch, conv_channels, new_seq_len]
        # Flatten the output for the LSTM input if necessary
        x = x.permute(0, 2, 1)  # Change back to [batch, length, channels]
        x = x.reshape(batch_size, -1, self.lstm_input_size)


        # LSTM processing
        if self.hn is None or self.cn is None:
            self.hn, self.cn = self.init_hidden(batch_size)
        else:
            self.hn, self.cn = self.hn.detach(), self.cn.detach()
            last_hn = self.hn[:,-1:,:]
            last_cn = self.cn[:,-1:,:]

            self.hn = last_hn.repeat(1, x.size(0), 1)
            self.cn = last_cn.repeat(1, x.size(0), 1)
        
        x, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))
        x = self.dropout_2(x[:, -1, :])  # Only take the last time step
        x = self.fc(x)

        return x

In [301]:
def find_supported_splits(comp_size, num_companies):
    supported_splits = []
    for n_splits in range(1, comp_size + 1):
        if comp_size % n_splits != 0:
            continue
        
        supported_splits.append(n_splits)
    return supported_splits


def custom_time_series_folds(data, n_splits):

    total_size = len(data)
    comp_size = total_size // len(company_list)
    comp_fold_size = comp_size//n_splits

    if comp_size % n_splits != 0:
        supported_splits = find_supported_splits(comp_size, n_splits)
        print(supported_splits)
        print(f"fold_size: {comp_fold_size} comp_size: {comp_size}")
        raise ValueError("Fold size must be divisible by the number of companies.")

    accumulated_train_idx = []     

    for i in range(n_splits-1):
        current_fold_val_idx = []
        current_fold_train_idx = []

        for j in range(len(company_list)):

            start_idx = j * comp_size
            val_start_idx = start_idx + (i+1) * comp_fold_size 
        
            end_idx = val_start_idx + comp_fold_size
        
            current_comp_train_idx = list(range(start_idx, val_start_idx))
            current_fold_train_idx.extend(current_comp_train_idx)  
        
            val_idx = list(range(val_start_idx, end_idx))
            current_fold_val_idx.extend(val_idx)  

        
        yield current_fold_train_idx, current_fold_val_idx


In [302]:
class ModelActioner:

    def __init__(self, train_data, test_data, device):
        self.train_data = train_data
        self.test_data = test_data
        self.device = device
        self.model = None
        self.optimizer = None
        self.criterion = nn.MSELoss()

    def custom_time_series_cv(self, config, trial):
        batch_size = config["batch_size"]
        epochs = config["epochs"]
        hidden_size = config["hidden_size"]
        num_layers = config["num_layers"]
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = epochs//5
        gamma = config["gamma"]
        kernel_size = config["kernel_size"]
        conv_channels = config["conv_channels"]
        # pool_size = config["pool_size"]
        # stride = config["stride"]
 

        suffle = False

        fold_results = []
        num_of_fold = 4

        for fold, (train_idx, val_idx) in enumerate(custom_time_series_folds(self.train_data, num_of_fold)):
            print(f"Fold: {fold+1}/{num_of_fold}")

            train_subset = Subset(self.train_data, train_idx)
            val_subset = Subset(self.train_data, val_idx)

            train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=suffle)
            val_loader = DataLoader(val_subset, batch_size=3, shuffle=suffle)

            self.model = OneDimCNNLSTMModel(input_dim=self.train_data.__getitem__(0)[0].shape[1], hidden_size=hidden_size, layer_size=num_layers, dropout_prob=dropout_prob, output_dim=1, conv_channels=conv_channels, kernel_size=kernel_size).to(self.device)

            self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
            scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min") 

            for epoch in range(epochs):
                print('epochs {}/{}'.format(epoch+1,epochs))

                self.model.reset_hidden()

                running_loss = 0.0
                total_sample_train = 0

                self.model.train()

                for batch_idx, (data, target) in enumerate(train_loader):
                    data, target = data.to(self.device), target.to(self.device)
                    target = target.view(-1,1) 

                    self.optimizer.zero_grad()
                    preds = self.model(data)

                    loss = self.criterion(preds, target)
                    loss.backward()
                    self.optimizer.step() # Update model params

                    running_loss += loss.item() * data.size(0)
                    total_sample_train += data.size(0)

                train_loss = running_loss/total_sample_train

                self.model.reset_hidden()
                self.model.eval()

                val_running_loss = 0.0
                total_sample_val = 0

                with torch.no_grad():

                    for batch_idx, (data, target) in enumerate(val_loader):
                        data, target = data.to(self.device), target.to(self.device)
                        target = target.view(-1,1)

                        preds = self.model(data)
                        loss = self.criterion(preds, target)

                        val_running_loss += loss.item() * data.size(0)
                        total_sample_val += data.size(0)
                
                val_loss = val_running_loss/total_sample_val
                fold_results.append(val_loss)
                scheduler.step(train_loss)
                
                unique_step = fold * epochs + epoch
                trial.report(val_loss, unique_step)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                current_lr = self.optimizer.param_groups[0]['lr']

                print(f'Current Learning Rate: {current_lr}')
                print(f"train_loss: {train_loss}, val_loss: {val_loss}")
                
        mean_val_loss = np.mean(fold_results)
        print(f"Mean validation loss: {mean_val_loss}")
        return mean_val_loss


                    
    def train(self, config):
        batch_size = config["batch_size"]
        epochs = config["epochs"]
        hidden_size = config["hidden_size"]
        num_layers = config["num_layers"]
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = epochs//5
        gamma = config["gamma"]
        kernel_size = config["kernel_size"]
        conv_channels = config["conv_channels"]
        # pool_size = config["pool_size"]
        # stride = config["stride"]
 
        self.model = OneDimCNNLSTMModel(input_dim=self.train_data.__getitem__(0)[0].shape[1], hidden_size=hidden_size, layer_size=num_layers, dropout_prob=dropout_prob, output_dim=1, conv_channels=conv_channels, kernel_size=kernel_size).to(self.device)

        # Update optimizer with updated lr
        self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate, weight_decay=weight_decay)

        # Creating data loader
        train_loader = DataLoader(dataset=self.train_data, batch_size=batch_size, shuffle=False)

        scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min")  

        # Training Loop
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))

            self.model.reset_hidden()
            
            running_loss = 0.0
            total_sample_train = 0

            self.model.train()

            for batch_idx, (data, target) in enumerate(train_loader):

                data, target = data.to(self.device), target.to(self.device)
                target = target.view(-1,1)  
                # print(data)

                self.optimizer.zero_grad()
                preds = self.model(data)

                loss = self.criterion(preds, target)
                loss.backward()
                self.optimizer.step() # Update model params

                running_loss += loss.item() * data.size(0)
                total_sample_train += data.size(0)

            train_loss = running_loss/total_sample_train
            scheduler.step(train_loss)
            current_lr = self.optimizer.param_groups[0]['lr']

            print(f'Current Learning Rate: {current_lr}')
            print(f"train_loss: {train_loss}")
        
        return self.model
            
    
    def test(self, config):
        batch_size = config["batch_size"]
        all_preds = []

        test_loader = DataLoader(dataset=self.test_data, batch_size=batch_size, shuffle=False)

        running_loss = .0
        total_sample = 0

        self.model.reset_hidden()
        self.model.eval()

        with torch.no_grad():

            for batch_idx, (data, target) in enumerate(test_loader):

                data, target = data.to(self.device), target.to(self.device)
                target = target.view(-1,1)
                
                preds = self.model(data)
                loss = self.criterion(preds, target)

                running_loss += loss.item() * data.size(0)
                total_sample += data.size(0)

                all_preds.extend(preds.cpu().numpy())

            test_loss = running_loss/total_sample
            print(f"test_loss: {test_loss}")

        return all_preds
    


In [303]:
def objective(trial):
    config = {
        "batch_size": trial.suggest_int("batch_size", 22, 100),
        "epochs": trial.suggest_int("epochs", 100, 400),
        "hidden_size": trial.suggest_int("hidden_size", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1),
        "dropout_prob": trial.suggest_float("dropout_prob", 0.1, 0.2),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True),
        # "lr_step_size": trial.suggest_int("lr_step_size", 3, 10), 
        "gamma": trial.suggest_float("gamma", 1e-4, 1e-1),
        "conv_channels": trial.suggest_int("conv_channels", 16, 128, step=16),
        "kernel_size": trial.suggest_int("kernel_size", 1, 3),
        "num_layers": trial.suggest_int("num_layers", 1, 5),
        # "pool_size": trial.suggest_int("pool_size", 2, 6),
        # "stride": trial.suggest_int("stride", 1, 3)
    }

    trainer = ModelActioner(train_data, test_data, device)

    val_loss = trainer.custom_time_series_cv(config, trial)

    return val_loss

In [304]:
study_name = "CNN-LSTM-Tunner"
storage_url = "sqlite:///db.sqlite3"

storage = optuna.storages.RDBStorage(url=storage_url)

# Check if the study exists
study_names = [study.study_name for study in optuna.study.get_all_study_summaries(storage=storage)]
if study_name in study_names:
    # Delete the study if it exists
    print(f"Deleting study '{study_name}'")
    optuna.delete_study(study_name=study_name, storage=storage_url)
else:
    print(f"Study '{study_name}' does not exist in the storage.")
    
study = optuna.create_study(direction='minimize', 
                            storage=storage_url, 
                            sampler=TPESampler(),
                            pruner=optuna.pruners.SuccessiveHalvingPruner(
                            min_resource=3,  # Minimum amount of resource allocated to a trial
                            reduction_factor=3,  # Reduction factor for pruning
                            min_early_stopping_rate=2  # Minimum early-stopping rate
                            ),
                            study_name=study_name,
                            load_if_exists=False)

pbar = tqdm(total=20, desc='Optimizing', unit='trial')

def callback(study, trial):
    # Update the progress bar
    pbar.update(1)
    pbar.set_postfix_str(f"Best Value: {study.best_value:.4f}")

study.optimize(objective, n_trials=20, callbacks=[callback])
pbar.close()

# Best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value:', trial.value)
print('Params:')
for key, value in trial.params.items():
    print(f'{key}: {value}')

Deleting study 'CNN-LSTM-Tunner'


[I 2024-05-07 20:24:24,556] A new study created in RDB with name: CNN-LSTM-Tunner


Optimizing:   0%|          | 0/20 [00:00<?, ?trial/s]

Fold: 1/4
epochs 1/213
Current Learning Rate: 0.039528154285592344
train_loss: 94.34097599127779, val_loss: 44.479259411493935
epochs 2/213
Current Learning Rate: 0.039528154285592344
train_loss: 5.5508590166767435, val_loss: 0.05576068742892756
epochs 3/213
Current Learning Rate: 0.039528154285592344
train_loss: 1.5640971969675135, val_loss: 110.03163115183513
epochs 4/213
Current Learning Rate: 0.039528154285592344
train_loss: 21.374660855090177, val_loss: 0.15888623146990236
epochs 5/213
Current Learning Rate: 0.039528154285592344
train_loss: 1.1646797905365627, val_loss: 5.125060634480582
epochs 6/213
Current Learning Rate: 0.039528154285592344
train_loss: 1.5848159315409485, val_loss: 2.719027812282244
epochs 7/213
Current Learning Rate: 0.039528154285592344
train_loss: 0.6479871598658739, val_loss: 0.6776986455337869
epochs 8/213
Current Learning Rate: 0.039528154285592344
train_loss: 0.8347158418209465, val_loss: 1.07033560383651
epochs 9/213
Current Learning Rate: 0.03952815428

[I 2024-05-07 20:39:43,454] Trial 0 finished with value: 1.022800977753545 and parameters: {'batch_size': 28, 'epochs': 213, 'hidden_size': 879, 'learning_rate': 0.039528154285592344, 'dropout_prob': 0.12381526884264604, 'weight_decay': 0.014200215743600426, 'gamma': 0.09634732843572635, 'conv_channels': 48, 'kernel_size': 1, 'num_layers': 3}. Best is trial 0 with value: 1.022800977753545.


...

Fold: 1/4
epochs 1/136
Current Learning Rate: 0.038331715800234825
train_loss: 29.701419876474475, val_loss: 102.0684724383884
epochs 2/136
Current Learning Rate: 0.038331715800234825
train_loss: 48.83374382389916, val_loss: 40.71386660469903
epochs 3/136
Current Learning Rate: 0.038331715800234825
train_loss: 28.637286754118072, val_loss: 20.322111950980293
epochs 4/136
Current Learning Rate: 0.038331715800234825
train_loss: 21.925101942486233, val_loss: 3.306044105026457
epochs 5/136
Current Learning Rate: 0.038331715800234825
train_loss: 5.529387543598811, val_loss: 5.770918246772554
epochs 6/136
Current Learning Rate: 0.038331715800234825
train_loss: 3.663276395863957, val_loss: 4.306446767515606
epochs 7/136
Current Learning Rate: 0.038331715800234825
train_loss: 2.735299896862772, val_loss: 0.11721709664197988
epochs 8/136
Current Learning Rate: 0.038331715800234825
train_loss: 1.3167514387104247, val_loss: 0.16402855717589976
epochs 9/136
Current Learning Rate: 0.038331715800234

[I 2024-05-07 20:49:53,834] Trial 18 pruned. 


Fold: 1/4
epochs 1/201
Current Learning Rate: 0.08176533434746597
train_loss: 232.2129563478477, val_loss: 28.50268472565545
epochs 2/201
Current Learning Rate: 0.08176533434746597
train_loss: 10.182199745542473, val_loss: 13.965817530949911
epochs 3/201
Current Learning Rate: 0.08176533434746597
train_loss: 9.193273135909328, val_loss: 0.05606358045932009
epochs 4/201
Current Learning Rate: 0.08176533434746597
train_loss: 6.690628426218474, val_loss: 0.1698375774988866
epochs 5/201
Current Learning Rate: 0.08176533434746597
train_loss: 2.588465573611083, val_loss: 3.658511483007007
epochs 6/201
Current Learning Rate: 0.08176533434746597
train_loss: 4.684486942158805, val_loss: 0.593368940676252
epochs 7/201
Current Learning Rate: 0.08176533434746597
train_loss: 3.1524761578551046, val_loss: 12.87155372566647
epochs 8/201
Current Learning Rate: 0.08176533434746597
train_loss: 6.511892365084754, val_loss: 5.180139382680257
epochs 9/201
Current Learning Rate: 0.08176533434746597
train_lo

[I 2024-05-07 20:50:08,889] Trial 19 pruned. 


Number of finished trials: 20
Best trial:
Value: 0.04417854509752057
Params:
batch_size: 29
epochs: 136
hidden_size: 401
learning_rate: 0.002173913346612086
dropout_prob: 0.13670260827829842
weight_decay: 5.971344548682018e-05
gamma: 0.0167966935710767
conv_channels: 80
kernel_size: 1
num_layers: 5


In [305]:
model = ModelActioner(train_data=train_data,test_data=test_data,device=device)
model.train(trial.params)

epochs 1/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.12517318265357366
epochs 2/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.06719497934035543
epochs 3/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.06747081076200293
epochs 4/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.05622061065191196
epochs 5/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.04823265260482138
epochs 6/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.03796529760303976
epochs 7/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.03044320971713643
epochs 8/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.03286694636227804
epochs 9/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.03259271110356268
epochs 10/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.029458158706633927
epochs 11/136
Current Learning Rate: 0.002173913346612086
train_loss: 0.026877352926897368
epochs 12/136
Cur

OneDimCNNLSTMModel(
  (conv): Conv1d(57, 80, kernel_size=(1,), stride=(1,), padding=(2,))
  (relu1): ReLU()
  (dropout_1): Dropout(p=0.13670260827829842, inplace=False)
  (lstm): LSTM(80, 401, num_layers=5, batch_first=True, dropout=0.13670260827829842)
  (dropout_2): Dropout(p=0.13670260827829842, inplace=False)
  (fc): Linear(in_features=401, out_features=1, bias=True)
)

In [306]:
y_test = y_test.reshape(-1,1)
print(len(y_test))
y_test[:4]

252


array([[1.10666667],
       [1.02666667],
       [1.        ],
       [1.26666667]])

In [307]:
preds = model.test(trial.params)

test_loss: 0.10558876643578212


In [308]:
preds = np.array(preds)
len(preds)

252

In [309]:
preds_inverse = []
y_true_inverse = []

idx = 0
data_size = len(y_test)//len(company_list)
print(data_size)

preds_comp_data = []
y_true_comp_data = pd.DataFrame()

for company in company_list:
    scaler = company_dict[company]["scaler_y"]
    preds_inverse.append(scaler.inverse_transform(preds[idx:idx+data_size]))
    y_true_inverse.append(scaler.inverse_transform(y_test[idx:idx+data_size].reshape(-1, 1) ))

    for pred in preds_inverse:
        preds_comp_data.append({"Prediction": pred, "Company": company})

    idx += data_size



# for company in company_list:
#     preds_inverse.append(np.array(preds).flatten())
#     y_true_inverse.append(np.array(y_test).flatten())
comp_preds_inverse = np.array(preds_inverse)
comp_true_inverse = np.array(y_true_inverse)

preds_inverse = np.array(preds_inverse).flatten()
y_true_inverse = np.array(y_true_inverse).flatten()
print(y_true_inverse.shape)
print(preds_inverse.shape)



mse = mean_squared_error(y_true_inverse, preds_inverse)
mape = mean_absolute_percentage_error(y_true_inverse, preds_inverse)*100
mae = mean_absolute_error(y_true_inverse, preds_inverse)
r2 = r2_score(y_true_inverse,preds_inverse)

print(f"MAPE Score: %{mape:.2f}")
print(f"MSE Score: {mse:.2f}")
print(f"MAE Score: {mae:.2f}")
print(f"R_2 Score: {r2:.2f}")

7
(252,)
(252,)
MAPE Score: %26.87
MSE Score: 0.18
MAE Score: 0.31
R_2 Score: 0.90


In [310]:
preds_inverse

array([0.9491113 , 0.9583211 , 0.9650893 , 0.96887094, 0.9699886 ,
       0.9703193 , 0.97093767, 2.4993455 , 2.5532386 , 2.5811653 ,
       2.609108  , 2.6206524 , 2.6246848 , 2.623246  , 1.8702964 ,
       1.8973935 , 1.9187646 , 1.9329811 , 1.9376363 , 1.937757  ,
       1.9388676 , 0.9296104 , 0.9423028 , 0.95085865, 0.95306396,
       0.9524521 , 0.9478674 , 0.94055223, 0.9763765 , 0.98598534,
       0.99351346, 0.9981543 , 0.9989512 , 0.9988447 , 0.99884194,
       5.9994636 , 6.0601487 , 6.104047  , 6.1319346 , 6.134231  ,
       6.1354184 , 6.136641  , 1.0138965 , 1.0412184 , 1.0661298 ,
       1.0846885 , 1.086496  , 1.0836338 , 1.0837128 , 1.1505904 ,
       1.1641707 , 1.1760508 , 1.1951197 , 1.1948907 , 1.1908303 ,
       1.1922711 , 1.3580722 , 1.376837  , 1.3773673 , 1.3907557 ,
       1.396161  , 1.3968805 , 1.3966826 , 1.5870882 , 1.6149808 ,
       1.6361946 , 1.6563485 , 1.660127  , 1.6583315 , 1.6607614 ,
       2.2883408 , 2.3127513 , 2.3325222 , 2.3433917 , 2.34578

In [311]:
len(y_true_inverse)

252

In [312]:
comp_preds_inverse

array([[[0.9491113 ],
        [0.9583211 ],
        [0.9650893 ],
        [0.96887094],
        [0.9699886 ],
        [0.9703193 ],
        [0.97093767]],

       [[2.4993455 ],
        [2.5532386 ],
        [2.5811653 ],
        [2.609108  ],
        [2.6206524 ],
        [2.6246848 ],
        [2.623246  ]],

       [[1.8702964 ],
        [1.8973935 ],
        [1.9187646 ],
        [1.9329811 ],
        [1.9376363 ],
        [1.937757  ],
        [1.9388676 ]],

       [[0.9296104 ],
        [0.9423028 ],
        [0.95085865],
        [0.95306396],
        [0.9524521 ],
        [0.9478674 ],
        [0.94055223]],

       [[0.9763765 ],
        [0.98598534],
        [0.99351346],
        [0.9981543 ],
        [0.9989512 ],
        [0.9988447 ],
        [0.99884194]],

       [[5.9994636 ],
        [6.0601487 ],
        [6.104047  ],
        [6.1319346 ],
        [6.134231  ],
        [6.1354184 ],
        [6.136641  ]],

       [[1.0138965 ],
        [1.0412184 ],
        [1.0661298 ]

In [313]:
y_true_inverse

array([1.06  , 1.03  , 1.02  , 1.12  , 1.13  , 1.16  , 1.03  , 1.71  ,
       1.3   , 2.16  , 3.31  , 1.87  , 1.45  , 2.53  , 1.84  , 1.88  ,
       1.74  , 2.01  , 2.07  , 2.08  , 2.03  , 0.75  , 1.13  , 0.54  ,
       0.72  , 0.65  , 0.61  , 0.7   , 1.03  , 1.06  , 0.99  , 0.94  ,
       1.01  , 1.07  , 1.01  , 6.66  , 7.52  , 6.08  , 6.61  , 6.41  ,
       7.15  , 8.34  , 0.9   , 0.76  , 0.05  , 0.44  , 0.57  , 0.58  ,
       0.72  , 1.05  , 1.06  , 1.54  , 1.11  , 1.13  , 1.29  , 1.21  ,
       1.87  , 1.95  , 1.88  , 2.11  , 1.94  , 1.96  , 1.85  , 1.74  ,
       1.45  , 1.77  , 1.62  , 1.76  , 1.9   , 1.52  , 3.16  , 2.99  ,
       1.87  , 2.9   , 3.07  , 3.17  , 2.25  , 5.98  , 6.28  , 6.04  ,
       5.71  , 5.81  , 4.79  , 4.69  , 1.13  , 1.15  , 0.78  , 1.39  ,
       1.33  , 1.5   , 1.43  , 0.66  , 0.66  , 0.44  , 0.56  , 0.65  ,
       0.7   , 0.64  , 0.44  , 0.45  , 0.48  , 0.4   , 0.4   , 0.48  ,
       0.48  , 1.66  , 1.78  , 1.55  , 1.78  , 1.89  , 2.15  , 1.96  ,
      

In [314]:
preds_inverse = np.array(preds_inverse).flatten()
preds_inverse

array([0.9491113 , 0.9583211 , 0.9650893 , 0.96887094, 0.9699886 ,
       0.9703193 , 0.97093767, 2.4993455 , 2.5532386 , 2.5811653 ,
       2.609108  , 2.6206524 , 2.6246848 , 2.623246  , 1.8702964 ,
       1.8973935 , 1.9187646 , 1.9329811 , 1.9376363 , 1.937757  ,
       1.9388676 , 0.9296104 , 0.9423028 , 0.95085865, 0.95306396,
       0.9524521 , 0.9478674 , 0.94055223, 0.9763765 , 0.98598534,
       0.99351346, 0.9981543 , 0.9989512 , 0.9988447 , 0.99884194,
       5.9994636 , 6.0601487 , 6.104047  , 6.1319346 , 6.134231  ,
       6.1354184 , 6.136641  , 1.0138965 , 1.0412184 , 1.0661298 ,
       1.0846885 , 1.086496  , 1.0836338 , 1.0837128 , 1.1505904 ,
       1.1641707 , 1.1760508 , 1.1951197 , 1.1948907 , 1.1908303 ,
       1.1922711 , 1.3580722 , 1.376837  , 1.3773673 , 1.3907557 ,
       1.396161  , 1.3968805 , 1.3966826 , 1.5870882 , 1.6149808 ,
       1.6361946 , 1.6563485 , 1.660127  , 1.6583315 , 1.6607614 ,
       2.2883408 , 2.3127513 , 2.3325222 , 2.3433917 , 2.34578

In [315]:
comp_true_inverse

array([[[1.06  ],
        [1.03  ],
        [1.02  ],
        [1.12  ],
        [1.13  ],
        [1.16  ],
        [1.03  ]],

       [[1.71  ],
        [1.3   ],
        [2.16  ],
        [3.31  ],
        [1.87  ],
        [1.45  ],
        [2.53  ]],

       [[1.84  ],
        [1.88  ],
        [1.74  ],
        [2.01  ],
        [2.07  ],
        [2.08  ],
        [2.03  ]],

       [[0.75  ],
        [1.13  ],
        [0.54  ],
        [0.72  ],
        [0.65  ],
        [0.61  ],
        [0.7   ]],

       [[1.03  ],
        [1.06  ],
        [0.99  ],
        [0.94  ],
        [1.01  ],
        [1.07  ],
        [1.01  ]],

       [[6.66  ],
        [7.52  ],
        [6.08  ],
        [6.61  ],
        [6.41  ],
        [7.15  ],
        [8.34  ]],

       [[0.9   ],
        [0.76  ],
        [0.05  ],
        [0.44  ],
        [0.57  ],
        [0.58  ],
        [0.72  ]],

       [[1.05  ],
        [1.06  ],
        [1.54  ],
        [1.11  ],
        [1.13  ],
        [1.29 

In [316]:
len(y_test)

252

In [317]:
y_test.flatten()
x_indices = np.arange(len(y_true_inverse))
preds = np.array(preds)
preds

array([[0.8109635 ],
       [0.835523  ],
       [0.85357153],
       [0.8636559 ],
       [0.8666363 ],
       [0.86751807],
       [0.8691671 ],
       [0.79357255],
       [0.81720984],
       [0.8294585 ],
       [0.841714  ],
       [0.8467773 ],
       [0.8485459 ],
       [0.8479148 ],
       [0.81171834],
       [0.83392906],
       [0.8514464 ],
       [0.86309934],
       [0.866915  ],
       [0.86701393],
       [0.8679242 ],
       [0.7897916 ],
       [0.8193089 ],
       [0.8392062 ],
       [0.84433484],
       [0.84291196],
       [0.83224976],
       [0.81523776],
       [0.8182007 ],
       [0.8323314 ],
       [0.84340215],
       [0.8502269 ],
       [0.8513988 ],
       [0.8512422 ],
       [0.85123813],
       [0.81898284],
       [0.83466375],
       [0.846007  ],
       [0.8532131 ],
       [0.8538065 ],
       [0.85411334],
       [0.85442924],
       [0.792963  ],
       [0.8088479 ],
       [0.82333124],
       [0.8341212 ],
       [0.83517206],
       [0.833

In [318]:
x_indices

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [319]:
pivoted_data.shape[1]

61

In [320]:
len(y_test)//len(company_list)

7

In [321]:
len(y_test)

252

In [322]:
x_indices = [["2022-Q2", "2022-Q3" ,"2022-Q4", '2023-Q1', '2023-Q2', '2023-Q3']] * len(company_list)

In [323]:
x_indices

[['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2', '2023-Q3'],
 ['2022-Q2', '2022-Q3', '2022-Q4',

In [324]:
# trace_true = go.Scatter(x=x_indices, y=y_true_inverse, mode="lines+markers", name=f"All Stocks True: EPS - PENDS")
# trace_preds = go.Scatter(x=x_indices, y=preds_inverse, mode="lines+markers", name=f"All Stocks Preds: EPS - PENDS")


stock_symbol = "Multiple Companies"

# Initialize the figure with layout
layout = go.Layout(
    title=f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)
fig = go.Figure(layout=layout)

# Variables to keep track of the extended x-axis and y-axis data
extended_x_indices = []
all_true_y = []
all_preds_y = []

# Adjust x_indices and concatenate y data for each company
for i, company in enumerate(company_list):
    # Extend x_indices with company identifier to differentiate time points between companies
    extended_x_indices += [f"{x} {company}" for x in x_indices[i]]
    all_true_y += list(comp_true_inverse[i].flatten())
    all_preds_y += list(comp_preds_inverse[i].flatten())

# Add traces for the concatenated true and predicted EPS values
trace_true = go.Scatter(x=extended_x_indices, y=all_true_y, mode="lines+markers", name="All Companies True: EPS - PENDS")
trace_preds = go.Scatter(x=extended_x_indices, y=all_preds_y, mode="lines+markers", name="All Companies Preds: EPS - PENDS")

# Add traces to the figure
fig.add_trace(trace_true)
fig.add_trace(trace_preds)

# Show the figure
fig.show()

In [325]:
pivoted_data[pivoted_data["OFTIC"] == "RCL"]["EPS"].tail(3)

Series([], Name: EPS, dtype: float64)

In [326]:
trace_true = go.Scatter(x=x_indices, y=y_true_inverse, mode="lines+markers", name=f"All Stocks: EPS - PENDS")
trace_preds = go.Scatter(x=x_indices, y=preds_inverse, mode="lines+markers", name=f"All Stocks: EPS - PENDS")


layout = go.Layout(
    title = f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=[trace_true, trace_preds], layout=layout)
fig.show()